In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
from utils.helper import fetch_all_metrics

load_dotenv()
PASSWORD = os.getenv("PGPASSWORD")
USER = os.getenv("PGUSER")
DB = os.getenv("PGDATABASE")
PORT = os.getenv("PGPORT")
HOST = os.getenv("PGHOST")
MODE = os.getenv("PGSSLMODE")

In [ ]:
CONNECTION = f"timescaledb://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}?sslmode={MODE}"
DATA_SAVE_PATH = "/Users/taimourabdulkarim/Desktop/Real Time Machine Telemetry Dashboard/data/raw_data"

In [ ]:
# Create engine and connect to the database
engine = create_engine(CONNECTION)

# Define the query
query = "SELECT * FROM weather;"

with engine.connect() as conn:
    df = pd.read_sql(query, conn)

df.head()

,time,city,city_id,condition_id,condition_main,country,forecast,host,cloudiness,condition_description,...,humidity,pressure,rain,snow,sunrise,sunset,temperature,visibility,wind_degrees,wind_speed
0,2024-11-20 04:09:08,Lahore,1172451,711,Smoke,PK,*,6a3ae9fe3c07,3,smoke,...,77,1017.0,0.0,0.0,1732066507000000000,1732104106000000000,13.99,2000,0.0,0.00
1,2024-11-20 04:09:08,Karachi,1174872,721,Haze,PK,*,6a3ae9fe3c07,0,haze,...,53,1017.0,0.0,0.0,1732067515000000000,1732106584000000000,22.90,2500,70.0,3.09
2,2024-11-20 04:05:28,Islamabad,1162015,800,Clear,PK,*,6a3ae9fe3c07,0,clear sky,...,61,1017.0,0.0,0.0,1732067060000000000,1732104134000000000,16.58,10000,299.0,0.49
3,2024-11-20 04:09:08,Kharian,1174042,800,Clear,PK,*,6a3ae9fe3c07,2,clear sky,...,28,1017.0,0.0,0.0,1732066774000000000,1732104070000000000,18.50,10000,334.0,1.39
4,2024-11-20 04:09:08,Murree,1169684,800,Clear,PK,*,6a3ae9fe3c07,0,clear sky,...,21,1017.0,0.0,0.0,1732067022000000000,1732104048000000000,6.80,10000,16.0,2.12


In [ ]:
df.shape

In [ ]:
df.to_csv(f"{DATA_SAVE_PATH}/DB_weather_data.csv", index=False)

In [ ]:
df = pd.read_csv(
    "/Users/taimourabdulkarim/Desktop/Real Time Machine Telemetry Dashboard/data/raw_data/DB_weather_data.csv"
)
df.shape

In [ ]:
df.info()

In [ ]:
df["time"] = pd.to_datetime(df["time"])
df = df.sort_values("time", ascending=False)

In [ ]:
cities = df["city"].unique()
cities = cities.tolist()

aggreate_metrics = {}
for city in cities:
    metrics = fetch_all_metrics(city, df)
    aggreate_metrics.update(metrics)

aggreate_metrics

In [ ]:
import json

with open(f"{DATA_SAVE_PATH}/aggreate_metrics.json", "w") as f:
    json.dump(aggreate_metrics, f, indent=4)